gachilabs / gachilabs123

## 연습문제로 실력 굳히기
인스타그램에 자동으로 로그인 한 후 '강남맛집추천'으로 검색하여 나오는 게시물 중 50개를 추출하여 사진과 해시태그를 수집하여 사진은 사진 폴더에 별도로 저장하고 (사진파일의 이름은 1.jpg부터 50.jpg) 해시태그는 txt파일 형식으로 저장하는 웹 크롤러를 저장하세요. (단 게시물의 사진이 여러 장 일 경우 대표사진 1장만 수집하여 저장하면 됩니다)

In [36]:
##########################################################################
# 인스타 그램의 해쉬태그 수집하기 - by 서진수
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver

import urllib.request
import urllib

import time
import math
import os
import random
import unicodedata   # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.
print("=" *70)
print("   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다")
print("   본 제품은 서진수가 교육용으로 특별 제작했으며 ")
print("   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다")
print("   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^")
print("=" *70)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 10)

f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : e:\BigAI\\ ) : ')
if f_dir =='' :
    f_dir = "e:\BigAI\\"

#Step 3. 결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성합니다.
s_time = time.time( )
query_txt2 = '인스타그램'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

# Step 4. 인스타그램 자동 로그인 하기
chrome_path = "c:/Users/aa/Downloads/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
time.sleep(random.randrange(1,5))

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

time.sleep(random.randrange(1,5))
driver.find_element_by_class_name('KPnG0').click()

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('email')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('pass')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_id('loginbutton').click()
time.sleep(10)

#Step 5. 검색할 해쉬태그 입력하기
#로그인 정보 나중에 저장하기
driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
time.sleep(3)

driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[1]/div').click()
time.sleep(2)

# 검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(5)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div/div[2]/div/div[1]/a/div').click()

import pandas as pd

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)
    
print('요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^')
item=[]     # 인스타그램 URL 주소 저장할 리스트
item2=[]
    
a = 1
while (a <= real_cnt):
    scroll_down(driver) 

    # Step 6. 전체 게시물의 원본 URL 추출하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    all = soup.find('article','KC1QD').find_all('a')

    for i in all:    
        url = i['href']
        item.append(url)          
        item2.append(pd.Series(item).drop_duplicates()) # 중복되는 값 삭제
    
        if len(item2) >= cnt :
            break
    a += 1
    print('요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^')
    
    if len(item2) >= cnt :
        break
   
# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 0
for x in range(0,len(item2)) :
    url = 'https://www.instagram.com' +item[x]
    full_url.append(url)
    url_cnt += 1
    
    if url_cnt > cnt:
        break

# 아래 for 반복문은 수집된 URL 주소를 확인하는 코드임        
for c in range(1,len(full_url)+1) :
   print(c,':',full_url[c-1])

#Step 7. 각 페이지별로 그림과 해쉬태그를 수집하기
count = 1        # 추출 데이터 건수 세기
hash_txt = []    # 해쉬 태그 저장 

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
import sys
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)
# 특이한 이모티콘은 변환해서 저장을 해준다.

count = 0

os.makedirs(f_dir+s+'-'+query_txt2+'-'+query_txt)
f_name=f_dir+s+'-'+query_txt2+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'

file_no = 0                                
count = 1
img_src2 = []

for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    f = open(f_name, 'a',encoding='UTF-8')

    img_src = soup.find('div','KL4Bh').find('img')['src']
    print(img_src)
    img_src2.append(img_src)
        

    # 해당 페이지의 해시태그 수집
    tags = soup.find('div','EtaWk')

    try :
        tags_1 = tags.find_all('a')
    except :
        pass
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            # 특별한 이모티콘을 일반적인 글자로 변화시켜라
            tags_11 = tags.translate(bmp_map)
            # 자음과 모음이 깨지지 않게 하기위해 변환 과정을 거친다.
            tags_2 = unicodedata.normalize('NFC', tags_11)

            for i in tags_2 :
                if i[0:1]=='#' :
                    hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    f.close()
    count += 1


os.makedirs(f_dir+s+'-'+query_txt2+'-'+'image')
os.chdir(f_dir+s+'-'+query_txt2+'-'+'image')

for i in range(0,len(img_src2)) :
        try :
                urllib.request.urlretrieve(img_src2[i],str(file_no)+'.jpg')
        except :
                continue        
        file_no += 1                
        time.sleep(0.5)      
        print("%s 번째 이미지 저장중입니다=======" %file_no)
    
#Step 7. 요약 정보 출력하기    
e_time = time.time( )
t_time = e_time - s_time

print("=" *100)
print("총 소요시간: %s 초" %round(t_time,1))
print("총 저장 건수: %s 건 " %count)
print("파일 저장 경로: %s" %f_name)
print("=" *100)

driver.close( )

   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다
   본 제품은 서진수가 교육용으로 특별 제작했으며 
   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다
   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^
1.인스타그램의 ID를 입력하세요: 01071646177
2.인스타그램의 비밀번호를 입력하세요: wjstjdgh*1
3.검색할 해쉬태그를 입력하세요(예: 강남맛집): 강남맛집
4.크롤링 할 건수는 몇건입니까?: 50
5.파일이 저장될 경로만 쓰세요(기본경로 : e:\BigAI\ ) : 


요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^


요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^
요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^
요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^
1 : https://www.instagram.com/p/CRQGdklF0Ww/
2 : https://www.instagram.com/p/CROfIHul-Qe/
3 : https://www.instagram.com/p/CROSs04ndbt/
4 : https://www.instagram.com/p/CRLpS9xlzbX/
5 : https://www.instagram.com/p/CRNoK_KF_ZR/
6 : https://www.instagram.com/p/CROcEG5HuKu/
7 : https://www.instagram.com/p/CRRKQQLFJ63/
8 : https://www.instagram.com/p/CRL6RrDl8vi/
9 : https://www.instagram.com/p/CRObyOclKdX/
10 : https://www.instagram.com/p/CRS8GXQDCJ4/
11 : https://www.instagram.com/p/CRS7kXDl_lP/
12 : https://www.instagram.com/p/CRS7i-KsUNa/
13 : https://www.i

https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/217428025_1175300066271724_4449263080953287171_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=84x5uwX2_OQAX9oFBBh&edm=AABBvjUBAAAA&ccb=7-4&oh=69fc8e390820ff24e20d9be04a587e88&oe=60F5CA88&_nc_sid=83d603
#원조부안집교대역점
#서울교대맛집
#고기스타그램
#강남맛집추천
#서초맛집
#교대맛집
#교대맛집원조부안집
#교대역맛집
#강남맛집
#서울데이트
#서울맛집
#서울고기집
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/217406825_177975500981884_606075180084557294_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=ML73EpCCRisAX_mFQrv&edm=AABBvjUBAAAA&ccb=7-4&oh=2111acb3e2d5e04aa345a84301142bcc&oe=60F4B1C0&_nc_sid=83d603
#압구정
#압구정로데오
#압구정맛집
#분위기맛집
#신사맛집
#컨셉비
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/p1080x1080/214296832_856998791573752_2734814472375273032_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=104&_nc_ohc=3zJyJkQeE0EAX9YkDuH&edm=AABBvjUBAAAA&ccb=7-4&oh=5a3aeeeef3392b96f307f3cc557d2ac5&oe=60F623BE&_nc_sid=83d603
https://scon

https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/fr/e15/s1080x1080/211517837_1480183695733884_6114877609692085333_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=100&_nc_ohc=iTc4bBK-sVQAX-WPsVl&edm=AABBvjUBAAAA&ccb=7-4&oh=083a3b0deac44a7f75a4515833d20caf&oe=60F5274C&_nc_sid=83d603
#석산정
#서울맛집
#서울맛집추천
#서울데이트
#서울데이트코스
#서울핫플
#서울핫플레이스
#서울가볼만한곳
#을지로맛집
#을지로핫플
#서울여행
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/213682246_338918804429495_7960076835792259637_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=111&_nc_ohc=aMeXHL6v44QAX9unPIT&edm=AABBvjUBAAAA&ccb=7-4&oh=64f311b2251265c2d5e33cc6ceaf7694&oe=60F551B1&_nc_sid=83d603
#막국수
#송림면옥
#비빔막국수
#물막국수
#백숙찜닭
#죽
#도가니닭곰탕
#김치맛집
#초계탕
#갈비찜
#서울
#서울맛집
#강남
#강남맛집
#역삼동
#역삼동맛집
#역삼역맛집
#먜슐랭
#먜슐랭_강남
#먜슐랭_역삼동
#문썬과함께
https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e35/s1080x1080/213847341_175474537832149_7583279716866474488_n.jpg?_nc_ht=scontent-ssn1-1.cdninstagram.com&_nc_cat=1&_nc_ohc=DtZnyd7FybgAX-LOdE5&edm=AABBvjUB

유투브에서 특정 동영상 검색 후 리뷰 정보 수집 후  저장하기-영상 요약정보 + 리뷰

In [ ]:
# 유투브에서 검색 후 동영상의 리뷰 정보 수집하기
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver

import time
import sys
import re
import math
import numpy 
import pandas as pd   
import xlwt 
import random
import os
#import openpyxl
# import urllib.request
# import urllib

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print("유투브 영상의 댓글을 수집하는 웹크롤러입니다")
print("=" *80)

query_txt=input("1.유튜브에서 검색할 주제 키워드를 입력하세요(예:올리브영): ")

movie_cnt = int(input('2.위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?:'))
reple_cnt = int(input('3.각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_dir = input("4.크롤링 결과를 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir = 'c:\\temp\\'

print("\n")
print("요청하신 정보로 데이터를 수집중이니 잠시만 기다려 주세요~~^^")   

 # Step 3. 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+'youtube'+'-'+s+'-'+query_txt)
os.chdir(f_dir+'youtube'+'-'+s+'-'+query_txt)

ff_dir=f_dir+'youtube'+'-'+s+'-'+query_txt
ff_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+'youtube'+'-'+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

s_time = time.time( )

# Step 4. 크롬 드라이버 설정하고 유투브 사이트에 접속하여 검색을 진행합니다 
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver= webdriver.Chrome()

driver.get('https://www.youtube.com')

time.sleep(5)
element = driver.find_element_by_name("search_query")
element.send_keys(query_txt)
element.submit()
time.sleep(2)

# Step 5. 검색할 동영상의 URL 정보를 수집하여 대상 영상 목록을 만듭니다.
# 검색 결과가 첫 화면에 20개가 나옵니다.
# 만약 사용자가 입력한 동영상 개수가 20개가 넘어갈 경우 스크롤다운해서 갯수를
# 먼저 만든 후 URL 을 추출하여 URL 을 완성해야 합니다

def scroll_down(driver):
    #driver.execute_script("window.scrollTo(500,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,3000);") # 한페이지 20개씩 출력값
    time.sleep(5)

if movie_cnt < 20 :
    page_cnt = 1 
else :
    page_cnt = math.ceil(cnt/20)

if movie_cnt > 20 :
    i = 1
    while ( i <=  page_cnt) :
        print("화면을 %s 회 아래로 스크롤 합니다" %i)
        scroll_down(driver)
        time.sleep(1)
        i += 1

# 검색된 유튜브 영상의 URL 을 추출합니다

time.sleep(2)

html = driver.page_source
soup1 = BeautifulSoup(html, 'html.parser')

count = 0
item=[]

print("\n")

for i in soup1.find_all('a','yt-simple-endpoint style-scope ytd-video-renderer'):
    item.append(i['href'])
    count += 1

    if count == movie_cnt :
        break

print("검색된 영상 중 %s 건 동영상의 댓글을 추출하겠습니다" %movie_cnt)
print("\n")

# Step 6. 동영상의 상세 정보를 수집합니다.
# 수집된 전체 URL 중에서 사용자가 입력한 수만큼의 동영상의 댓글을 수집합니다.
# 먼저 전체 URL 을 완성 -> 동영상 페이지 열기 -> 댓글 수집 -> 다음 동영상 순으로 진행합니다
full_url=[]
url_cnt = 0
for x in range(0,len(item)) :
    url_cnt += 1
    url = 'https://www.youtube.com/' +item[x]
    full_url.append(url)
    if url_cnt == movie_cnt :
        break

play_cnt = 1

url2=[]
reple2=[]
reple3=[]
reple4=[]
writer2=[]
wdate2=[]
good2=[]
bad2=[]

real_collect_count = 0   #실제 총 수집한 리플 갯수
# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

for y in range(0,len(full_url)) :
    driver.get(full_url[y])
    time.sleep(10)
    print("\n")
    print("\n")
    print("%s 번째 동영상의 정보를 수집합니다." %play_cnt)

    i = 1
    while ( i <=  page_cnt) :
        scroll_down(driver)
        time.sleep(3)
        i += 1

    html = driver.page_source
    soup2 = BeautifulSoup(html, 'html.parser')
    
    # 조회수
    t_count_0 = soup2.select('#count')
    t_count_1 = t_count_0[1].get_text()
    t_count_2 = t_count_1.replace(",","")
    t_count_3 = re.search("\d+",t_count_2)
    t_count_4 = int(t_count_3.group())
    
    # 제목
    t_title_1 = soup2.select('#info-contents')
    t_title_2 = t_title_1[0].find('h1').get_text()
    t_title_3 = t_title_2.translate(bmp_map).replace("\n","")
    
    # 댓글수
    t_view_1 = soup2.find('yt-formatted-string','count-text style-scope ytd-comments-header-renderer').get_text()#.get_text( ).replace("\n","")
    t_view_2 = t_view_1.replace("," ,"")
    t_view_3 = re.search("\d+",t_view_2)
    t_view_4 = int(t_view_3.group())
    
    #좋아요 / 싫어요 수
    t_good_bad = soup2.select('#top-level-buttons-computed')
    t_good = t_good_bad[0].select('#text')[0].get_text()  # 좋아요 수
    t_bad = t_good_bad[0].select('#text')[1].get_text()   # 싫어요 수

    print("=" *120)
    print("%s 번째 동영상의 제목은 %s 입니다" %(play_cnt,t_title_3))
    print("%s 번째 동영상의 조회수는 %s 회, 댓글수는 총 %s개, 좋아요는 %s개, 싫어요는 %s개입니다" %(play_cnt,t_count_4,t_view_4,t_good,t_bad))  
    print("=" *120)
    print("\n")
        
    #댓글 정보를 수집하기 위해 화면을 스크롤해서 아래로 이동합니다
    if t_view_4 > 0 :
        print("%s 번째 동영상에서 댓글 수집을 시작합니다" %play_cnt)
        print("댓글의 개수가 많아서 시간이 걸릴 수 있으니 잠시 기다려 주세요~~^^")
        print("\n")
        if reple_cnt < 20 :
            reple_page_cnt = 1
        else :
            reple_page_cnt = math.ceil(reple_cnt / 20)

        i = 1
        while ( i <  reple_page_cnt) :
            print("%s 번째 페이지의 댓글을 수집하는 중입니다~" %i)
            scroll_down(driver)
            time.sleep(0.5)
            i += 1

        #내용을 수집합니다

        time.sleep(2)

        html = driver.page_source
        soup3 = BeautifulSoup(html, 'html.parser')

        count = 0

        reple_result = soup3.select('#comments > #sections > #contents')

        for a in range(0,reple_cnt+1) :
            real_collect_count += 1
            count += 1

            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
            f.write("------------------------------------------------------------------"+"\n")

            # 댓글 작성자
            try : 
                writer = reple_result[0].select(" #author-text")[a].get_text().replace("\n","").strip()
            except  :
                break
            else :
                print("\n")
                print("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
                print("-" *70)

                # 유튜브 URL 주소
                print("1.URL 주소:",full_url[y])
                f.write("1.유투브 URL주소: " + full_url[y] + "\n")
                url2.append(full_url[y])

                print("2.댓글 작성자명:", writer)
                f.write("2.댓글 작성자명: " + writer + "\n")
                writer2.append(writer)

                # 댓글 작성일자
                wdate = reple_result[0].select("yt-formatted-string.published-time-text > a.yt-simple-endpoint")[a].get_text().replace("\n","")

                print("3.댓글 작성일자:",wdate)
                f.write("3.댓글 작성일자: " + wdate + "\n")
                wdate2.append(wdate)

                # 댓글 내용
                reple1 = reple_result[0].select('#expander > #content > #content-text')[a].get_text().replace("\n","")

                reple2=reple1.translate(bmp_map).replace("\n","")
                print("4.댓글 내용:",reple2)
                f.write("4.댓글 내용: " + reple2 + "\n")
                reple3.append(reple2)

                f.write("\n")

                #좋아요 / 싫어요
                good2.append(t_good)
                bad2.append(t_bad)
                
            f.close( )

            if count == reple_cnt :
                break

        time.sleep(2)       
        play_cnt += 1  
    else :
        print("%s 번째 동영상의 댓글수가 %s개이므로 다음영상으로 넘어갑니다" %(play_cnt,t_view_4) )
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("%s 번째 영상의 %s 번째  댓글 " %(play_cnt,count))
        f.write("------------------------------------------------------------------"+"\n")
        f.write("%s 번째 동영상의 댓글수가 %s개이므로 다음영상으로 넘어갑니다" %(play_cnt,t_view_4) )
        f.write("\n")
        f.close( )
        
        play_cnt += 1 

 #step 9. 엑셀 형태로 저장하기
                
youtube_reple = pd.DataFrame()
youtube_reple['URL 주소']=url2
youtube_reple['댓글작성자명']=pd.Series(writer2)
youtube_reple['댓글작성일자']=pd.Series(wdate2)
youtube_reple['댓글 내용']=pd.Series(reple3)
youtube_reple['좋아요수']=pd.Series(good2)
youtube_reple['싫어요수']=pd.Series(bad2)

# csv 형태로 저장하기
youtube_reple.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
youtube_reple.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

real_collect_count -= 1

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %real_collect_count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

print("\n") 
print("=" *100)
print("1.요청된 총 %s 건 동영상에서 각 영상별 실제 크롤링 된 리뷰수는 총 %s 건입니다" %(movie_cnt,real_collect_count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

driver.close( )

# 쿠팡 사이트 식품 카테고리 베스트상품

이미지와 정보를 exel로 가져오기

In [1]:
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd    
import os
import math

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print(" 쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기 ")
print("=" *80)

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt/60)

f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본경로:e:\BigAI\\):")
if f_dir == '' :
    f_dir = "e:\BigAI\\"
    
print("\n")

if cnt > 30 :
      print("    요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")

#Step 3.저장될 파일 경로와 이름을 지정합니다
sec_name = '식품'
query_txt='쿠팡'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xls'

# 제품 이미지 저장용 폴더 생성
img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)
    
s_time = time.time( )

#Step 4. 웹사이트 접속 후 해당 메뉴로 이동합니다.
chrome_path = "c:/Users/aa/Downloads/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
query_url='https://www.coupang.com/'
driver.get(query_url)
time.sleep(5)

driver.delete_all_cookies() #쿠키 제거


# 분야별 더보기 버튼을 눌러 페이지를 엽니다
driver.find_element_by_xpath("""//*[@id="header"]/div""").click( )
driver.find_element_by_xpath("""//*[@id="gnbAnalytics"]/ul[1]/li[4]/a""").click( )

#Step 5. 내용을 수집합니다
print("\n")
print("===== 곧 수집된 결과를 출력합니다 ^^ ===== ")
print("\n")

ranking2=[]        #제품의 판매순위 저장
title2=[]          #제품 정보 저장
p_price2=[]        #현재 판매가 저장
discount2 = []     #할인율 저장
sat_count2=[]      #상품평 수 저장

img_src2=[]   # 이미지 URL 저장변수
file_no = 0   # 이미지 파일 저장할 때 번호
count = 1     # 총 게시물 건수 카운트 변수

def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)

scroll_down(driver)   #현재화면의 가장 아래로 스크롤다운합니다

for x in range(1,page_cnt + 1) :
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    item_result = soup.find('ul','baby-product-list').find_all('li')

    for li in item_result :
        if cnt < count :
            break

        # 제품 이미지 다운로드 하기
        import urllib.request
        import urllib

        try :
            photo = li.find('dt','image').find('img')['src']
        except AttributeError :
            continue

        file_no += 1
        full_photo = 'https:' + photo
        urllib.request.urlretrieve(full_photo,str(file_no)+'.jpg')
        time.sleep(0.5)

        #제품 내용 추출하기
        f = open(ff_name, 'a',encoding='UTF-8')
        f.write("-----------------------------------------------------"+"\n")
        print("-" *70)

        ranking = count
        print("1.판매순위:",ranking)
        f.write('1.판매순위:'+ str(ranking) + "\n")

        try :
            t = li.find('div','name').get_text().replace("\n","")
        except :
            title = '제품소개가 없습니다'
            print(title.replace("\n",""))
            f.write('2.제품소개:'+ title + "\n")
        else :
            title = t.replace("\n","").strip()
            print("2.제품소개:", title.replace("\n","").strip())                  
            f.write('2.제품소개:'+ title + "\n")

        try :
            p_price = li.find('strong','price-value').get_text().replace("\n","")
        except :
            p_price = '0'
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")
        else :
            print("3.판매가격:", p_price.replace("\n",""))
            f.write('3.판매가격:'+ p_price + "\n")

        try :
            discount = li.find('span','discount-percentage').get_text().replace("\n","")
        except  :
            discount = '0'
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")
        else :
            print("4:할인률:", discount)
            f.write('4.할인율:'+ discount + "\n")

        try :
            sat_count_1 = li.find('span','rating-total-count').get_text()
            sat_count_2 = sat_count_1.replace("(","").replace(")","")
        except  :
            sat_count_2='0'
            print('5.상품평 수: ',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")
        else :
            print('5.상품평 수:',sat_count_2)
            f.write('5.상품평 수:'+ sat_count_2 + "\n")

        print("-" *70)

        f.close( )             
        time.sleep(0.5)

        ranking2.append(ranking)
        title2.append(title.replace("\n",""))

        p_price2.append(p_price.replace("\n",""))
        discount2.append(discount)

        try :   
            sat_count2.append(sat_count_2)
        except IndexError :
            sat_count2.append(0)

        count += 1
    x += 1       
    try :
        driver.find_element_by_link_text('%s' %x).click() # 다음 페이지번호 클릭
    except :
        pass
          
#step 6. csv , xls 형태로 저장하기              
co_best_seller = pd.DataFrame()
co_best_seller['판매순위']=ranking2
co_best_seller['제품소개']=pd.Series(title2)
co_best_seller['제품판매가']=pd.Series(p_price2)
co_best_seller['할인율']=pd.Series(discount2)
co_best_seller['상품평수']=pd.Series(sat_count2)

# csv 형태로 저장하기
co_best_seller.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
co_best_seller.to_excel(fx_name ,index=False)

e_time = time.time( )
t_time = e_time - s_time

count -= 1
print("\n")
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

#Step 7. xls 파일에 제품 이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api                   #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     
excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(2).ColumnWidth = 30   
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120   

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
      col_name='B'+str(a)
      col_name2.append(col_name)

for b in range(1,cnt+1) :
      file_name=img_dir+'\\'+str(b)+'.jpg'
      file_name2.append(file_name)
      
for i in range(0,cnt) :
      rng = ws.Range(col_name2[i])
      image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
      excel.Visible=True
      excel.ActiveWorkbook.Save()

driver.close( )

 쿠팡 사이트의 식품 카테고리 Best Seller 상품 정보 추출하기 
1.크롤링 할 건수는 몇건입니까?: 10
2.파일을 저장할 폴더명만 쓰세요(기본경로:e:\BigAI\):


    요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~


===== 곧 수집된 결과를 출력합니다 ^^ ===== 


----------------------------------------------------------------------
1.판매순위: 1
2.제품소개: 홍성 홍산마늘, 2kg, 1박스
3.판매가격: 21,900
4:할인률: 0
5.상품평 수: 4
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 2
2.제품소개: 캐나다산 활랍스터, 900g, 1개
3.판매가격: 47,400
4:할인률: 24%
5.상품평 수: 100
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 3
2.제품소개: 제주 삼다수, 2L, 24개
3.판매가격: 23,490
4:할인률: 40%
5.상품평 수: 303388
----------------------------------------------------------------------
----------------------------------------------------------------------
1.판매순위: 4
2.제품소개: 가야산 천년수 무라벨, 2L, 24개
3.판매가격: 9,900
4:할인률: 0
5.상품평 수: 23528
---------------------------

### 헤드리스 모드 사용

In [ ]:
option = webdriver.ChromeOptions()
options.add_argument('headless')
driver = wevdriver.Chrome(chrome_path, option=options)

### 실행 파일로 변경하는 방법

pyinstaller 사용 - cmd mode (CLI)

pyQT5 - window mode (GUI)